In [1]:
from transformers import AutoTokenizer, AutoModel

model_llm = "Writer/camel-5b-hf"
model_emb = "BAAI/bge-small-en-v1.5"

save_llm = "../models/camel-5b-hf"
save_emb = "../models/bge-small-en-v1.5"

tokenizer = AutoTokenizer.from_pretrained(model_llm)
model = AutoModel.from_pretrained(model_llm)
tokenizer.save_pretrained(save_llm)
model.save_pretrained(save_llm)

tokenizer = AutoTokenizer.from_pretrained(model_emb)
model = AutoModel.from_pretrained(model_emb)
tokenizer.save_pretrained(save_emb)
model.save_pretrained(save_emb)

/opt/anaconda3/envs/rag_llamaindex/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 3/3 [00:00<00:00, 11.52it/s]


In [18]:
from llama_index.llms.huggingface import HuggingFaceLLM


llm = HuggingFaceLLM(
    tokenizer_name = save_llm,
    model_name = save_llm,
    context_window = 2048,
    max_new_tokens = 512,
    generate_kwargs={"temperature": 0.25, "do_sample" : True}
    device_map = "auto",
    tokenizer_kwargs={"max_length" : 2048}
)

TypeError: transformers.models.auto.auto_factory._BaseAutoModelClass.from_pretrained() got multiple values for keyword argument 'device_map'

In [ ]:
llm.complete("What is GPT4")